# 文字信息提取

In [113]:
import re
import pandas as pd
file_path = "/Users/liangkaixi/Desktop/001.txt"  # 读取文件内容
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()
header_match = re.search(r'\n(.+?)\n', text)  # 提取第一行作为表头
header = header_match.group(1).strip()
members_data = []
for match in re.finditer(r'(\d+)\. (.+?)(?: (\S+))?$', text, re.MULTILINE):   # 通过正则表达式提取信息
    groups = match.groups()
    count = groups[2] if groups[2] is not None else '1'
    count_numeric_match = re.search(r'(\d+)', count)
    count_numeric = count_numeric_match.group() if count_numeric_match else '1'
    members_data.append(groups[:2] + (count_numeric,))
price = 78  # 写入单价
df = pd.DataFrame(members_data, columns=['序号', '昵称', header+f'-单价:{ price }'])  # 创建DataFrame
sum_column_3 = df[header+f'-单价:{ price }'].astype(int).sum()   # 计算第三列的和
df[header+'合计'] = df[header+f'-单价:{ price }'].astype(int)*price  # 写入合计金额
df.at[0, header + '_总件数'] = sum_column_3 # 将和写入第四列的第一个单元格
df['昵称'] = df['昵称'].str.strip()
file_name = header.replace(' ', '_')  # 将空格替换为下划线  
excel_file_path = f'/Users/liangkaixi/Desktop/水果调试/{file_name}.xlsx'   # 使用第一行文字作为Excel文件名
df.to_excel(excel_file_path, index=False)


# 统计汇总

In [134]:

import os
import pandas as pd
def process_xlsx_files(folder_path, output_excel_path):
    data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_excel(file_path)
            col_name = df.columns[4]                                  # 获取第4列的列名
            first_value = str(df.iloc[0, 4]) if pd.notna(df.iloc[0, 4]) else ""
            data.append({'类别': col_name, '总数量': first_value})
    output_df = pd.DataFrame(data)
    output_df.to_excel(output_excel_path, index=False)
if __name__ == "__main__":
    folder_path = "/Users/liangkaixi/Desktop/水果调试"                   # 请替换为你的文件夹路径
    output_excel_path = "/Users/liangkaixi/Desktop/output222.xlsx"      # 请替换为你想要输出的 Excel 文件路径
    process_xlsx_files(folder_path, output_excel_path)


# 数据merge

In [115]:
import os
import pandas as pd
folder_path = '/Users/liangkaixi/Desktop/水果调试'
file_list = [f for f in os.listdir(folder_path) if f.endswith('.xlsx') or f.endswith('.csv')]  # 获取文件夹中所有文件的列表
merged_data = pd.DataFrame(columns=['昵称'])    # 添加'昵称'列作为初始列
for file_name in file_list:                    # 循环遍历每个文件
    file_path = os.path.join(folder_path, file_name)   # 构建完整的文件路径
    df = pd.read_excel(file_path)                      # 读取文件
    df['昵称'] = df['昵称'].str.strip()                  # 去掉空格
    if '昵称' not in df.columns or df['昵称'].isnull().any():  # 检查是否存在"昵称"列和是否有缺失值
        continue
    selected_columns = df.iloc[:, 1:3]                        # 提取第2、3、4列数据
    merged_data = pd.merge(merged_data, selected_columns, on='昵称', how='outer')   # 按第2列进行merge

merged_data.to_excel('/Users/liangkaixi/Desktop/output_mergts.xlsx',index=False)


# 详情汇总透视

In [118]:
import pandas as pd
file_path = '/Users/liangkaixi/Desktop/output_mergts.xlsx'
df = pd.read_excel(file_path)
new_column_values = []
for index, row in df.iterrows():              # 按行遍历
   
    non_empty_values = row.iloc[1:].dropna()  # 排除第一列并检查该行下是否有值
    if not non_empty_values.empty:
        new_values = []
        total_amount = 0                      # 初始化总金额
        for column, value in non_empty_values.items():
            if value % 1 == 0:
                split_price = column.split(':')[-1]         # 切片找出需要的数据
                split_name = column.split('-')[0]
                item_amount = value * float(split_price)    
                total_amount += item_amount                 # 累加总金额
                new_values.append(f'{split_name}:{int(value)}件 {split_price}元/件')
        if total_amount > 0:
            formatted_total_amount = int(total_amount) if total_amount.is_integer() else round(total_amount, 1)  # 判断是否为整数
            new_values.append(f'总计:{formatted_total_amount}元')
        new_column_values.append(', '.join(new_values))                     # 将新值加入列表
    else:
        new_column_values.append('')                                        # 如果该行下没有值，则添加空字符串

df.insert(1, '订单详情列', new_column_values)                                 # 在第二列位置插入新列
df.to_excel('/Users/liangkaixi/Desktop/水果详情汇总表.xlsx', index=False)


In [136]:
import os
import pandas as pd
folder_path = '/Users/liangkaixi/Desktop/水果调试'
result_df = pd.DataFrame(columns=['类别', '总金额'])   # 创建一个空的DataFrame用于存储结果
for filename in os.listdir(folder_path):    # 遍历文件夹中的所有xlsx文件
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_excel(file_path)
        column_name = df.columns[3]  # 获取列名
        sum_result = df.iloc[:, 3].sum()   #  求和
        result_df = pd.concat([result_df, pd.DataFrame({'类别': [column_name], '总金额': [sum_result]})], ignore_index=True)
result_file_path = '/Users/liangkaixi/Desktop/各类别汇总.xlsx'
result_df.to_excel(result_file_path, index=False)


/var/folders/3f/zkzs2p554xncvph5tq3jbyd40000gn/T/ipykernel_17730/2288201186.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({'第4列列名': [column_name], '求和结果': [sum_result]})], ignore_index=True)
